# Data Chunking Basics

1. Read 1 year of 1 Degree surface pressure data from the UFS Replay
2. Store the data locally in two formats:
   1. 2D chunks, each chunk has a single timestep
   2. Time chunks, each chunk has all timesteps for a very small spatial region (5x5 grid cells)
3. Compute temporal and spatial averages and compare performance

## Data Setup

Start by opening the publicly available 1 Degree Replay dataset

In [ ]:
import xarray as xr

In [30]:
ds = xr.open_zarr(
    "gcs://noaa-ufs-gefsv13replay/ufs-hr1/1.00-degree/03h-freq/zarr/fv3.zarr",
    storage_options={"token": "anon"},
)

Select surface pressure during the first year, during 1994

In [32]:
ds = ds[["pressfc"]].sel(time=slice(None, "1994"))
ds.load();

### Form dataset with 2D chunks

In [34]:
ds.pressfc.encoding = {}
ds = ds.chunk({"time": 1, "grid_yt": -1, "grid_xt": -1})

In [35]:
ds.to_zarr("surface-pressure.2d-chunks.zarr", mode="w")

### Form dataset with long time chunks

In [37]:
ds.load();

In [38]:
ds.pressfc.encoding = {}
ds = ds.chunk({"time": -1, "grid_yt": 5, "grid_xt": 5})

In [40]:
ds.to_zarr("surface-pressure.time-chunks.zarr", mode="w")

## Open and View the Locally Stored Datasets 

Take note of:
- chunk size is about 290 kB for both datasets
- dask gives us a nice view of the chunking orientation
- there are about 3,000 chunks in each dataset

In [41]:
ds2d = xr.open_zarr("surface-pressure.2d-chunks.zarr")
dst = xr.open_zarr("surface-pressure.time-chunks.zarr")

In [42]:
ds2d.pressfc

<xarray.DataArray 'pressfc' (time: 2922, grid_yt: 192, grid_xt: 384)>
dask.array<open_dataset-pressfc, shape=(2922, 192, 384), dtype=float32, chunksize=(1, 192, 384), chunktype=numpy.ndarray>
Coordinates:
    cftime   (time) object dask.array<chunksize=(2922,), meta=np.ndarray>
    ftime    (time) timedelta64[ns] dask.array<chunksize=(2922,), meta=np.ndarray>
  * grid_xt  (grid_xt) float64 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
  * grid_yt  (grid_yt) float64 89.28 88.36 87.42 86.49 ... -87.42 -88.36 -89.28
  * time     (time) datetime64[ns] 1993-12-31T18:00:00 ... 1994-12-31T21:00:00
Attributes:
    cell_methods:  time: point
    long_name:     surface pressure
    output_file:   phy
    units:         Pa

In [43]:
dst.pressfc

<xarray.DataArray 'pressfc' (time: 2922, grid_yt: 192, grid_xt: 384)>
dask.array<open_dataset-pressfc, shape=(2922, 192, 384), dtype=float32, chunksize=(2922, 5, 5), chunktype=numpy.ndarray>
Coordinates:
    cftime   (time) object dask.array<chunksize=(2922,), meta=np.ndarray>
    ftime    (time) timedelta64[ns] dask.array<chunksize=(2922,), meta=np.ndarray>
  * grid_xt  (grid_xt) float64 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
  * grid_yt  (grid_yt) float64 89.28 88.36 87.42 86.49 ... -87.42 -88.36 -89.28
  * time     (time) datetime64[ns] 1993-12-31T18:00:00 ... 1994-12-31T21:00:00
Attributes:
    cell_methods:  time: point
    long_name:     surface pressure
    output_file:   phy
    units:         Pa

## Computations

### Compute spatial average for a single time step

This is 200-300x faster for the dataset chunked as 2D spatial slices

In [50]:
%time ds2d.isel(time=0).mean(["grid_xt", "grid_yt"]).compute();
%time dst.isel(time=0).mean(["grid_xt", "grid_yt"]).compute();

CPU times: user 7.62 ms, sys: 4.03 ms, total: 11.6 ms
Wall time: 10.2 ms
CPU times: user 2.86 s, sys: 783 ms, total: 3.64 s
Wall time: 2.74 s


### Compute time average for single point in space

This is 200-300x faster for the dataset with long time series chunks

In [54]:
%time ds2d.sel(grid_xt=200, grid_yt=0, method="nearest").mean("time").compute();
%time dst.sel(grid_xt=200, grid_yt=0, method="nearest").mean("time").compute();

CPU times: user 2.57 s, sys: 675 ms, total: 3.25 s
Wall time: 2.42 s
CPU times: user 6.66 ms, sys: 0 ns, total: 6.66 ms
Wall time: 7.25 ms


### Compute yearly time average at all points and spatial average at all time points

Because the chunksizes and number of chunks are about the same, these timings are about the same.

In [57]:
%time ds2d.mean("time").compute();
%time dst.mean("time").compute();

CPU times: user 3.16 s, sys: 750 ms, total: 3.91 s
Wall time: 2.24 s
CPU times: user 3.24 s, sys: 694 ms, total: 3.94 s
Wall time: 2.49 s


In [59]:
%time ds2d.mean(["grid_yt", "grid_xt"]).compute();
%time dst.mean(["grid_yt", "grid_xt"]).compute();

CPU times: user 2.92 s, sys: 769 ms, total: 3.69 s
Wall time: 2.5 s
CPU times: user 3.02 s, sys: 764 ms, total: 3.79 s
Wall time: 2.27 s
